# Index
1. ### [Content scraping](#content)
1. ### [Saving to CSV file](#export)

<a id='content'></a>

## Content scraping

In [141]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

def apostrophe(s):
    s = s.replace("\'", "'")
    return s

url = 'https://www.bbc.com/news/64642608'
result = requests.get(url)
src = result.content     
soup = BeautifulSoup(src, 'html.parser')


# extracting claim
claims = []
claims_html = soup.find_all('div', class_='ssrcss-dm4ypg-ComponentWrapper-HeadlineComponentWrapper egtrm1f0')
for i in claims_html:
    claim = i.text
    claim = claim.split(":")[1].strip().split("'")[1].strip().split("'")[0].strip()
    claims.append(claim)


# extracting justifying text
justifications = []
all_pars = []
paragraphs_html = soup.find_all('div', class_='ssrcss-7uxr49-RichTextContainer e5tfeyi1')
for i in paragraphs_html:
    all_pars.append(i.text)
    
to_remove = 0, 1, 22, 28, 29, 30
all_pars = [i for j, i in enumerate(all_pars) if j not in to_remove]

claim1 = all_pars[0:13]
claim2 = all_pars[13:17]
claim3 = all_pars[17:]
    
text1,text2,text3 = '','',''

for par in claim1:
    text1 += ' ' + apostrophe(par)
    text1 = text1.strip()
    
for par in claim2:
    text2 += ' ' + apostrophe(par)
    text2 = text2.strip()
    
for par in claim3:
    text3 += ' ' + apostrophe(par)
    text3 = text3.strip()
    
justifications.append(text1)
justifications.append(text2)
justifications.append(text3)


# extracting date
date_raw = soup.find('span', class_='ssrcss-1if1g9v-MetadataText ecn1o5v1').text
if len(date_raw.split()) == 2: date_raw += ' 2023'
date_object = datetime.strptime(date_raw, '%d %B %Y')
formatted_date = date_object.strftime('%d-%m-%Y')
dates = [formatted_date]*3


# extracting url
urls = ['https://www.bbc.com/news/64642608']*3


# extract label
veredicts = ['Misleading']*3

In [144]:
everything = list(zip(claims, veredicts, justifications, dates, urls))

<a id='export'></a>

## Saving to CSV file

In [145]:
import pandas as pd
df = pd.DataFrame.from_records(everything, columns=["Claim", "Label", "Evidence", "Date", "URL"])
pd.options.display.max_rows = None
df

,Claim,Label,Evidence,Date,URL
0,"While the poverty rate in India is about 16%, ...",Misleading,Mr Obi has taken data from two different sourc...,20-02-2023,https://www.bbc.com/news/64642608
1,"In just five years between 2015 and 2020, the ...",Misleading,"Mr Atiku Abubakar of the opposition PDP, who w...",20-02-2023,https://www.bbc.com/news/64642608
2,[Insecurity has] actually reduced... we [had] ...,Misleading,"Insecurity has been a major election issue, in...",20-02-2023,https://www.bbc.com/news/64642608


In [146]:
# save to csv file
df.to_csv("bbc.csv", index=False)